# Testing on datasets in mltk cache

In [35]:
import torch
import numpy as np
import datasets
import os
from datasets import Dataset
from torchvision.transforms import v2

os.chdir('/data/john/projects/mltoolkit/data')
os.listdir()

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Set cache directory and load dataset

In [81]:
# this is just to keep track of names
names = [
    'imagenet-1k',
    'cifar100',
    'pg19',
    'wikipedia',
]

# load the dataset
ds = datasets.load_dataset('imagenet-1k', 
                           trust_remote_code=True,
                           cache_dir='cache')

print(ds)
print(ds['train'][0]['image'])

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 1281167
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 100000
    })
})
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=817x363 at 0x7FEE95D30210>


# Make transform function and test

In [127]:
transform = v2.Compose([
    v2.Resize(size=(224,224), antialias=True),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
])

tfm = transform(ds['train'][0]['image'])
print(tfm.shape)
print(tfm)

torch.Size([3, 224, 224])
tensor([[[0.5529, 0.5529, 0.5569,  ..., 0.6471, 0.6353, 0.6275],
         [0.5529, 0.5608, 0.5647,  ..., 0.6471, 0.6392, 0.6353],
         [0.5569, 0.5608, 0.5569,  ..., 0.6549, 0.6471, 0.6471],
         ...,
         [0.7608, 0.7529, 0.7529,  ..., 0.6824, 0.6824, 0.6824],
         [0.7529, 0.7412, 0.7451,  ..., 0.6824, 0.6784, 0.6745],
         [0.7490, 0.7490, 0.7490,  ..., 0.6863, 0.6784, 0.6745]],

        [[0.5569, 0.5569, 0.5608,  ..., 0.6549, 0.6549, 0.6667],
         [0.5569, 0.5647, 0.5686,  ..., 0.6549, 0.6549, 0.6627],
         [0.5608, 0.5647, 0.5608,  ..., 0.6627, 0.6627, 0.6667],
         ...,
         [0.7569, 0.7529, 0.7529,  ..., 0.6941, 0.6941, 0.6980],
         [0.7569, 0.7569, 0.7608,  ..., 0.6980, 0.6941, 0.6902],
         [0.7490, 0.7569, 0.7608,  ..., 0.7020, 0.6941, 0.6902]],

        [[0.5765, 0.5765, 0.5804,  ..., 0.6745, 0.6706, 0.6745],
         [0.5765, 0.5843, 0.5843,  ..., 0.6745, 0.6706, 0.6784],
         [0.5804, 0.5843, 0.5765

# Do this

In [129]:
tfm = torch.stack([transform(sample) for sample in ds['train'][:2]['image']])

print(tfm.shape)

torch.Size([2, 3, 224, 224])


# This sucks ass

In [116]:
ds.set_transform(transform, 'image')
print(ds['train'][:10]['image'])
ds.reset_format()

[tensor([[[0.5529, 0.5529, 0.5569,  ..., 0.6471, 0.6353, 0.6275],
         [0.5529, 0.5608, 0.5647,  ..., 0.6471, 0.6392, 0.6353],
         [0.5569, 0.5608, 0.5569,  ..., 0.6549, 0.6471, 0.6471],
         ...,
         [0.7608, 0.7529, 0.7529,  ..., 0.6824, 0.6824, 0.6824],
         [0.7529, 0.7412, 0.7451,  ..., 0.6824, 0.6784, 0.6745],
         [0.7490, 0.7490, 0.7490,  ..., 0.6863, 0.6784, 0.6745]],

        [[0.5569, 0.5569, 0.5608,  ..., 0.6549, 0.6549, 0.6667],
         [0.5569, 0.5647, 0.5686,  ..., 0.6549, 0.6549, 0.6627],
         [0.5608, 0.5647, 0.5608,  ..., 0.6627, 0.6627, 0.6667],
         ...,
         [0.7569, 0.7529, 0.7529,  ..., 0.6941, 0.6941, 0.6980],
         [0.7569, 0.7569, 0.7608,  ..., 0.6980, 0.6941, 0.6902],
         [0.7490, 0.7569, 0.7608,  ..., 0.7020, 0.6941, 0.6902]],

        [[0.5765, 0.5765, 0.5804,  ..., 0.6745, 0.6706, 0.6745],
         [0.5765, 0.5843, 0.5843,  ..., 0.6745, 0.6706, 0.6784],
         [0.5804, 0.5843, 0.5765,  ..., 0.6824, 0.6784, 0